In [1]:
# calculate crop production
library(fields)
source('~/Desktop/analysis/code/get_geo.R')

Loading required package: spam
Loading required package: dotCall64
Loading required package: grid
Spam version 2.1-1 (2017-07-02) is loaded.
Type 'help( Spam)' or 'demo( spam)' for a short introduction 
and overview of this package.
Help for individual functions is also obtained by adding the
suffix '.spam' to the function name, e.g. 'help( chol.spam)'.

Attaching package: ‘spam’

The following objects are masked from ‘package:base’:

    backsolve, forwardsolve

Loading required package: maps


In [2]:
# FAO crop data, 0.5 degree resolution
load('~/Desktop/analysis/data/crop/crop.FAO-USDA.hres.0.5x0.5.1961-2010.RData')

In [3]:
ls()

[1] "ApplyOverCountry"    "ApplyOverState"      "CountryData2LonLat" 
 [4] "NaN.to.zero"         "UN.code.map"         "area.latlon"        
 [7] "crop.name"           "crop.prod"           "crop.yield"         
[10] "crop.yield.5yma"     "dist.latlon"         "find.Rlim"          
[13] "find.cor"            "find.lon.lat"        "find.region.2.5x2.5"
[16] "find.region.2x2.5"   "find.region.GISS"    "find.region.gcap"   
[19] "flip.lon"            "get.info.nc"         "harvest.area"       
[22] "inv.terrain.colors"  "invdist"             "lat.0.5"            
[25] "load.nc"             "lon.0.5"             "moran.I"            
[28] "neighbor.avg"        "par.uin"             "plot.field"         
[31] "plot.mod.obs.US"     "plot.nc"             "plot.site"          
[34] "plot.vprof"          "quick.image"         "quiver"             
[37] "rm.nonUS"            "rm.nonUS.2.5x2.5"    "rm.nonUS.2x2.5"     
[40] "rm.nonUS.GISS"       "rwb.colors"          "sp.dissolve"        
[43] "sp.dissolve.3D"      "sp.regrid"           "sp.regrid.3D"       
[46] "spavg"               "spsd"                "state.code.map"     
[49] "state.codes"         "unit_info"           "year.vec"           
[52] "zero.to.NaN"

In [4]:
year.vec

[1] 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975
[16] 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990
[31] 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005
[46] 2006 2007 2008 2009 2010

In [5]:
crop.name

[1] "wheat"   "rice"    "maize"   "soybean"

In [6]:
crop_prod = crop.yield*harvest.area # [lon, lat, years, crops]
dim(crop_prod)

[1] 720 360  50   4

In [7]:
crop_prod_5year_mean = apply(X = crop_prod[,,46:50,], FUN = mean, MARGIN = c(1,2,4), na.rm = T)
dim(crop_prod_5year_mean)

[1] 720 360   4

In [8]:
# lat and lon of TEMIR outputs
lat = seq(-90,90,2)
lon = seq(-180,177.5,2.5)
# output grid
prod_2degree = array(NaN, dim=c(length(lon),length(lat),length(crop.name)))
length(lon)
length(lat)

[1] 144

[1] 91

In [11]:
# split lon grids in half, for regridding
crop_prod_0.5x0.25 = array(NaN, dim=c(2*length(`lon.0.5`),length(`lat.0.5`),4))
lon.0.25 = seq(-179.875,179.875,0.25)
print(lon.0.5)
print(length(lon.0.25))
dim(crop_prod_0.5x0.25)
sum(crop_prod_5year_mean[,,3], na.rm = T) # global total prod. in tonnes
for(ilon in 1:(2*length(`lon.0.5`))){
    lon.ind = as.integer((ilon+1)/2)
    crop_prod_0.5x0.25[ilon,,] = 0.5*crop_prod_5year_mean[lon.ind,,]
    }
sum(crop_prod_0.5x0.25[,,3], na.rm = T)

  [1] -179.75 -179.25 -178.75 -178.25 -177.75 -177.25 -176.75 -176.25 -175.75
 [10] -175.25 -174.75 -174.25 -173.75 -173.25 -172.75 -172.25 -171.75 -171.25
 [19] -170.75 -170.25 -169.75 -169.25 -168.75 -168.25 -167.75 -167.25 -166.75
 [28] -166.25 -165.75 -165.25 -164.75 -164.25 -163.75 -163.25 -162.75 -162.25
 [37] -161.75 -161.25 -160.75 -160.25 -159.75 -159.25 -158.75 -158.25 -157.75
 [46] -157.25 -156.75 -156.25 -155.75 -155.25 -154.75 -154.25 -153.75 -153.25
 [55] -152.75 -152.25 -151.75 -151.25 -150.75 -150.25 -149.75 -149.25 -148.75
 [64] -148.25 -147.75 -147.25 -146.75 -146.25 -145.75 -145.25 -144.75 -144.25
 [73] -143.75 -143.25 -142.75 -142.25 -141.75 -141.25 -140.75 -140.25 -139.75
 [82] -139.25 -138.75 -138.25 -137.75 -137.25 -136.75 -136.25 -135.75 -135.25
 [91] -134.75 -134.25 -133.75 -133.25 -132.75 -132.25 -131.75 -131.25 -130.75
[100] -130.25 -129.75 -129.25 -128.75 -128.25 -127.75 -127.25 -126.75 -126.25
[109] -125.75 -125.25 -124.75 -124.25 -123.75 -123.25 -122.75 -1

[1] 1440  360    4

[1] 783018849

[1] 783018849

In [10]:
# before and after regridding
before = crop_prod_0.5x0.25
after = prod_2degree

for(icrop in 1:length(crop.name)){
    print(paste('global total production', crop.name[icrop], 'before regriding:', sum(before[,,icrop], na.rm = T)))

    for(ilat in 1:length(lat))
    for(ilon in 1:length(lon)){ # summing up grids belong to coarser grid
        
        # get the subset of data in finer grid that falls into coarser grid
        
        # 1, get lat indices
        if(ilat == 1) lat.ind = c(1,2) # first and last box contains only 2 finer grids
        else if(ilat == length(lat)) lat.ind = c(length(`lat.0.5`)-1,length(`lat.0.5`))
        # ilat = 2, lat.ind = 3,4,5,6, ilat = 3, lat.ind = 7,8,9,10
        else lat.ind = (3+(ilat-2)*4):(2+(ilat-1)*4)
        #print(lat.ind)
        
        # 2, get lon indices
        if(ilon == 1) lon.ind = c(1:5,(2*length(`lon.0.5`)-4):(2*length(`lon.0.5`))) 
        #ilon = 2, lon.ind = 6:15, ilon = 3, lon.ind = 16:25
        else lon.ind = (6+(ilon-2)*10):(5+(ilon-1)*10)
        
        subset = before[lon.ind,lat.ind,]
        after[ilon,ilat,icrop] = sum(subset[,,icrop], na.rm = T) # sum up the subset of data
    }
    #print(paste('global total production', crop.name[icrop], 'after regriding:', sum(after[,,icrop], na.rm = T)))
    #plot.field(after[,,i], lon, lat)
}
prod_2degree = after

[1] "global total production wheat before regriding: 592623510.164863"
[1] "global total production rice before regriding: 585787092.260699"
[1] "global total production maize before regriding: 783018849.145532"
[1] "global total production soybean before regriding: 246496717.554033"


In [12]:
crop.types = c('Wheat', 'Rice', 'Maize', 'Soybean')
for(i in 1:4) assign(x = paste0('prod_', crop.types[i]), value = prod_2degree[,,i])

setwd('~/Desktop/analysis/data/crop')
unit.info = 'crop.prod: ton/grid cell'
save(list = c('prod_Maize', 'prod_Rice', 'prod_Soybean', 'prod_Wheat', 'crop.types', 'unit.info'), file = 'crop_prod.RData')